In [1]:
!nvidia-smi

Wed Jul 27 09:57:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Atelier_Google_Colab/Projet_8

# List files to make sure we're in the expected directory.
!ls

/content/gdrive/MyDrive/Atelier_Google_Colab/Projet_8
checkpoint  data_redim	P08_01_notebook_modelisation.ipynb  saved  src


In [ ]:
#!pip uninstall imgaug
#!pip install git+https://github.com/aleju/imgaug.git

#### Imports

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import random
import cv2
import time

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU, OneHotMeanIoU

from keras.models import load_model

import imgaug as ia
import imgaug.augmenters as iaa

# Personal libraries
import src.datagenerator as dg
import src.models as models
import src.helpers as helpers

#### Constantes

In [22]:
# Images
leftImg8bit_path = 'data_redim/leftImg8bit/'
# Masques
gtFine_path = 'data_redim/gtFine/'

checkpoint_path = 'saved/checkpoint/'
models_path = 'saved/models/'
history_path = 'saved/history/'

if not os.path.isdir('saved'):
  os.makedirs(checkpoint_path)
  os.makedirs(models_path)
  os.makedirs(history_path)

DIM_X = 256
DIM_Y = 256
INPUT_SHAPE = (DIM_X,DIM_Y,3)
NB_CLASSES = 8

monitor_val = 'val_mean_iou'
learning_rate = 1e-4
nbr_epochs = 50
patience = 8

## Récupérer les chemins d'accès

In [13]:
# Recupère les chemins d'accès des fichiers dans le dossier précisé
def getPathFiles(dir_path, pattern = ""):

    train_path = dir_path + 'train/'
    test_path = dir_path + 'test/'
    val_path = dir_path + 'val/'

    train_path_files = []
    test_path_files = []
    val_path_files = []

    # Train set
    for dir in os.listdir(train_path):
        for file in os.listdir(train_path + dir):
            if pattern in file:
                train_path_files.append(train_path + dir + '/' +file)

    # Test set
    for dir in os.listdir(test_path):
        for file in os.listdir(test_path + dir):
            if pattern in file:
                test_path_files.append(test_path + dir + '/' +file)

    # Val set
    for dir in os.listdir(val_path):
        for file in os.listdir(val_path + dir):
            if pattern in file:
                val_path_files.append(val_path + dir + '/' +file)

    return train_path_files, test_path_files, val_path_files



In [15]:
# Listes des chemins des images
x_train_path, x_test_path, x_val_path = getPathFiles(leftImg8bit_path)
# Listes des chemins des masques
y_train_path, y_test_path, y_val_path = getPathFiles(gtFine_path, "labelIds.png")

# Echantillon préparé pour les tests
X_test, y_test = helpers.get_data_prepared(x_test_path[0:10], y_test_path[0:10], (DIM_X,DIM_Y))

## Redimensionner les images pour le stockage dans le Cloud

In [16]:
data_redim_path = 'data_redim'

list_dir = os.listdir()

if data_redim_path not in list_dir:
    os.mkdir('data_redim')

    for dir_1 in os.listdir('data'):
        if dir_1 != 'img':
            os.mkdir(data_redim_path + '/' + dir_1)
            for dir_2 in os.listdir('data' + '/' + dir_1):
                os.mkdir(data_redim_path + '/' + dir_1 + '/' + dir_2)
                for dir_3 in os.listdir('data' + '/' + dir_1 + '/' + dir_2):
                    os.mkdir(data_redim_path + '/' + dir_1 + '/' + dir_2 + '/' + dir_3)


    all_images_path = x_train_path + x_test_path + x_val_path + y_train_path + y_test_path + y_val_path

    for path in all_images_path:
        img = cv2.resize(cv2.imread(path), (1024, 512))
        new_path = path.replace('data', 'data_redim')
        cv2.imwrite(new_path, img)

else:
  print(f'Dossier {data_redim_path} déjà créé')

Dossier data_redim déjà créé


## Exemple data augmentation

In [17]:
X = np.array([cv2.resize(cv2.imread(path_X), (DIM_X, DIM_Y)) for path_X in x_train_path[0:5]])
y = np.array([cv2.resize(helpers.convert_mask(cv2.imread(path_y,0)), (DIM_X, DIM_Y)) for path_y in y_train_path[0:5]])

X_augmented, y_augmented = helpers.augment_data(X, y)
y_augmented_arg = np.argmax(y_augmented, axis=3)

In [18]:
for i in range(len(X)):
    plt.subplots(1, 4, figsize=(21,5))

    plt.subplot(1, 4, 1)
    plt.imshow(X[i])

    plt.subplot(1, 4, 2)
    plt.imshow(X_augmented[i])

    plt.subplot(1, 4, 3)
    plt.imshow(y_augmented_arg[i])

    plt.subplot(1, 4, 4)
    plt.imshow(X_augmented[i])
    plt.imshow(y_augmented_arg[i], alpha=0.3)

    plt.show()

Output hidden; open in https://colab.research.google.com to view.

## Data Generator

In [19]:
data_train = dg.DataGenerator(x_train_path, y_train_path, batch_size=20, dim=(DIM_X, DIM_Y), augmentation = False)
data_val = dg.DataGenerator(x_val_path, y_val_path, batch_size=20, dim=(DIM_X, DIM_Y), augmentation = False)

data_train_augmented = dg.DataGenerator(x_train_path, y_train_path, batch_size=10, dim=(DIM_X, DIM_Y), augmentation = True)
data_val_augmented = dg.DataGenerator(x_val_path, y_val_path, batch_size=10, dim=(DIM_X, DIM_Y), augmentation = True)

## Modèles

#### **Fonctions train & predict génériques**

In [20]:
import json

synthese = {}
train_times = {}
if os.path.exists('models/train_times.json'):
    with open('models/train_times.json', 'r') as f:
        train_times = json.load(f)

In [ ]:
def train_model(model, model_name, loss_function, augmented=False):
  filepath_check = checkpoint_path + model_name + '/'
  model_path = models_path + model_name
  history_path_file = history_path + model_name + '_history.json'

  # compile
  model.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss_function, metrics=[OneHotMeanIoU(num_classes=NB_CLASSES, name='mean_iou')])

  if os.path.exists(model_path):
    model = load_model(model_path, custom_objects=None, compile=True)
    print(f'Récupération du modèle {model.name}')

    training_time = train_times[model_name]

    with open(history_path_file, 'r') as f:
      history = json.load(f)
    hist_df = pd.DataFrame.from_dict(history)

  else:
    start_train = time.time()

    mc = ModelCheckpoint(mode='max', filepath=filepath_check, monitor=monitor_val, save_best_only='True', save_weights_only='True')
    es = EarlyStopping(mode='max', monitor=monitor_val, patience=8, verbose=1)

    callbacks = [mc, es]

    # fit
    if augmented:
      history = model.fit(data_train_augmented, epochs=nbr_epochs, validation_data=data_val_augmented, use_multiprocessing=True, workers=2, callbacks=callbacks)
    else:
      history = model.fit(data_train, epochs=nbr_epochs, validation_data=data_val, use_multiprocessing=True, workers=2, callbacks=callbacks)

    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history)

    # save history
    with open(history_path_file, mode='w') as f:
        hist_df.to_json(f)

    # Save the model
    model.load_weights(filepath_check)
    model.save(model_path)

    end_train = time.time()
    training_time = end_train - start_train
    train_times[model_name] = training_time

    with open(models_path + 'train_times.json', 'w') as f:
      json.dump(train_times, f)
  
  return model, hist_df, training_time

In [ ]:
def predict_model(model, model_name, loss_type, training_time, show_pred=False):
  loss_score, mean_iou_score = model.evaluate(data_val)
  print("Pour le meilleur modèle on obtient :")
  print("mean_iou :", mean_iou_score)
  print("loss :", loss_score)

  start_pred = time.time()

  y_pred=model.predict(X_test)
  y_pred_argmax=np.argmax(y_pred, axis=3)

  end_pred = time.time()
  predict_time = end_pred - start_pred

  if show_pred:
    for index in range(10):
        print(index)
        plt.imshow(X_test[index])
        plt.imshow(y_pred_argmax[index],alpha = 0.4)
        plt.show()

  synthese[model_name] = helpers.add_model_in_synthese(loss_type, loss_score, mean_iou_score, training_time, predict_time)

#### **Modèle Unet de base**

- Métrique Mean_iou
- Fonction loss 'categorical_crossentropy"
- Pas d'augmentation de données

In [ ]:
# Build model
model = models.build_unet(INPUT_SHAPE, NB_CLASSES)
model_name = model.name + '_base'
model.summary()

In [ ]:
# Train model
print(f'Modèle : {model_name}')
model, hist_df, training_time = train_model(model, model_name, "categorical_crossentropy", augmented=False)

In [ ]:
# Draw loss function and metric evolution
helpers.draw_history(hist_df)

In [ ]:
# Predict from model trained
predict_model(model, model_name, "categorical_crossentropy", training_time, show_pred=True)

#### **Optimisation de la fonction loss**

In [ ]:
from tensorflow.python.keras.losses import categorical_crossentropy
from tensorflow.python.keras import backend as K

def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.cast(K.flatten(y_true), K.floatx())
    y_pred_f = K.cast(K.flatten(y_pred), K.floatx())
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = categorical_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss

#### **Modèle Unet, fonction dice loss**

- Métrique Mean_iou
- Fonction loss 'dice loss"
- Pas d'augmentation de données

In [ ]:
model = models.build_unet(INPUT_SHAPE, NB_CLASSES)

model_name = model.name + '_dice_loss'
filepath_check = checkpoint_path + model_name + '/'
model_path = models_path + model_name
history_path_file = history_path + model_name + '_history.json'

# compile
model.compile(optimizer=Adam(learning_rate=learning_rate), loss=dice_loss, metrics=[OneHotMeanIoU(num_classes=8, name='mean_iou')])

if os.path.exists(model_path):
    model = load_model(model_path, custom_objects=None, compile=True)
    print(f'Récupération du modèle {model.name}')

    training_time = train_times[model_name]

    with open(history_path_file, 'r') as f:
      history = json.load(f)
    hist_df = pd.DataFrame.from_dict(history)

else:
  start_train = time.time()

  mc = ModelCheckpoint(mode='max', filepath=filepath_check, monitor=monitor_val, save_best_only='True', save_weights_only='True')
  es = EarlyStopping(mode='max', monitor=monitor_val, patience=8, verbose=1)

  callbacks = [mc, es]

  # fit
  history = model.fit(data_train, epochs=50, validation_data=data_val, use_multiprocessing=True, workers=2, callbacks=callbacks)

  # convert the history.history dict to a pandas DataFrame:     
  hist_df = pd.DataFrame(history.history)

  # save history
  with open(history_path_file, mode='w') as f:
      hist_df.to_json(f)

  # Save the model
  model.load_weights(filepath_check)
  model.save(model_path)

  end_train = time.time()
  training_time = end_train - start_train
  train_times[model_name] = training_time

  with open(models_path + 'train_times.json', 'w') as f:
    json.dump(train_times, f)

In [ ]:
loss_score, mean_iou_score = model.evaluate(data_val)
print("Pour le meilleur modèle on obtient :")
print("mean_iou :", mean_iou_score)
print("loss :", loss_score)

plt.style.use("default")

start_pred = time.time()

y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

end_pred = time.time()
predict_time = end_pred - start_pred

for index in range(10):
    print(index)
    plt.imshow(X_test[index])
    plt.imshow(y_pred_argmax[index],alpha = 0.4)
    plt.show()

synthese[model_name] = helpers.add_model_in_synthese("dice loss", loss_score, mean_iou_score, training_time, predict_time)

#### **Modèle Unet, fonction combinaison loss**

- Métrique Mean_iou
- Fonction loss 'combinaison loss" (dice loss & categorical crossentropy)
- Pas d'augmentation de données

In [ ]:
model = models.build_unet(INPUT_SHAPE, NB_CLASSES)

model_name = model.name + '_combine_loss'
filepath_check = checkpoint_path + model_name + '/'
model_path = models_path + model_name

mean_iou_metric.reset_states()

# compile
model.compile(optimizer=Adam(learning_rate=0.001), loss=total_loss, metrics=[mean_iou_metric])

if os.path.exists(model_path):
    model = load_model(model_path)
    training_time = train_times[model_name]

else:
  start_train = time.time()

  mc = ModelCheckpoint(mode='max', filepath=filepath_check, monitor=monitor_val, save_best_only='True', save_weights_only='True', verbose=1)
  es = EarlyStopping(mode='max', monitor=monitor_val, patience=4, verbose=1)

  callbacks = [mc, es]

  # fit
  history = model.fit(data_train, epochs=20, validation_data=data_val, use_multiprocessing=True, workers=2, callbacks=callbacks)

  # Save the model
  model.load_weights(filepath_check)
  model.save(model_path)

  end_train = time.time()
  training_time = end_train - start_train
  train_times[model_name] = training_time

  with open('models/train_times.json', 'w') as f:
    json.dump(train_times, f)

In [ ]:
loss_score, mean_iou_score = model.evaluate(data_val)
print("Pour le meilleur modèle on obtient :")
print("mean_iou :", mean_iou_score)
print("loss :", loss_score)

X_test, y_test = helpers.get_data_prepared(x_test_path[0:20], y_test_path[0:20], (256,256), n_classes=NB_CLASSES)

plt.style.use("default")

start_pred = time.time()

y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

end_pred = time.time()
predict_time = end_pred - start_pred

for index in range(10):
    print(index)
    plt.imshow(X_test[index])
    plt.imshow(y_pred_argmax[index],alpha = 0.4)
    plt.show()

synthese[model_name] = helpers.add_model_in_synthese("combine loss", loss_score, mean_iou_score, training_time, predict_time)

#### **Modèle Unet data augmented**

- Métrique Mean_iou
- Fonction loss 'combinaison loss"
- Augmentation de données

In [ ]:
model = models.build_unet(INPUT_SHAPE, NB_CLASSES)

model_name = model.name + '_augmented'
filepath_check = checkpoint_path + model_name + '/'
model_path = models_path + model_name
history_path_file = history_path + model_name + '_history.json'

# compile
model.compile(optimizer=Adam(learning_rate=learning_rate), loss=total_loss, metrics=[OneHotMeanIoU(num_classes=8, name='mean_iou')])

if os.path.exists(model_path):
    model = load_model(model_path, custom_objects=None, compile=True)
    print(f'Récupération du modèle {model.name}')

    training_time = train_times[model_name]

    with open(history_path_file, 'r') as f:
      history = json.load(f)
    hist_df = pd.DataFrame.from_dict(history)

else:
  start_train = time.time()

  mc = ModelCheckpoint(mode='max', filepath=filepath_check, monitor=monitor_val, save_best_only='True', save_weights_only='True')
  es = EarlyStopping(mode='max', monitor=monitor_val, patience=8, verbose=1)

  callbacks = [mc, es]

  # fit
  history = model.fit(data_train_augmented, epochs=50, validation_data=data_val_augmented, use_multiprocessing=True, workers=2, callbacks=callbacks)

  # convert the history.history dict to a pandas DataFrame:     
  hist_df = pd.DataFrame(history.history)

  # save history
  with open(history_path_file, mode='w') as f:
      hist_df.to_json(f)

  # Save the model
  model.load_weights(filepath_check)
  model.save(model_path)

  end_train = time.time()
  training_time = end_train - start_train
  train_times[model_name] = training_time

  with open('models/train_times.json', 'w') as f:
    json.dump(train_times, f)

softmax
Epoch 1/50
297/297 [==============================] - ETA: 0s - loss: 2.9505 - mean_iou: 0.1851
Epoch 00001: val_mean_iou improved from -inf to 0.17359, saving model to saved/checkpoint/U-Net_augmented/
297/297 [==============================] - 263s 865ms/step - loss: 2.9505 - mean_iou: 0.1851 - val_loss: 2.9069 - val_mean_iou: 0.1736
Epoch 2/50
297/297 [==============================] - ETA: 0s - loss: 2.6283 - mean_iou: 0.2166
Epoch 00002: val_mean_iou improved from 0.17359 to 0.20140, saving model to saved/checkpoint/U-Net_augmented/
297/297 [==============================] - 254s 853ms/step - loss: 2.6283 - mean_iou: 0.2166 - val_loss: 2.6927 - val_mean_iou: 0.2014
Epoch 3/50
297/297 [==============================] - ETA: 0s - loss: 2.5857 - mean_iou: 0.2220
Epoch 00003: val_mean_iou improved from 0.20140 to 0.20702, saving model to saved/checkpoint/U-Net_augmented/
297/297 [==============================] - 259s 867ms/step - loss: 2.5857 - mean_iou: 0.2220 - val_loss: 2.

In [ ]:
loss_score, mean_iou_score = model.evaluate(data_val)
print("Pour le meilleur modèle on obtient :")
print("mean_iou :", mean_iou_score)
print("loss :", loss_score)

X_test, y_test = helpers.get_data_prepared(x_test_path[0:20], y_test_path[0:20], (256,256), n_classes=NB_CLASSES)

plt.style.use("default")

start_pred = time.time()

y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

end_pred = time.time()
predict_time = end_pred - start_pred

for index in range(10):
    print(index)
    plt.imshow(X_test[index])
    plt.imshow(y_pred_argmax[index],alpha = 0.4)
    plt.show()

synthese[model_name] = helpers.add_model_in_synthese("combine loss", loss_score, mean_iou_score, training_time, predict_time)

Output hidden; open in https://colab.research.google.com to view.

#### **Modèle Resnet50 Unet de base**

- Métrique Mean_iou
- Fonction loss 'categorical_crossentropy"
- Pas d'augmentation de données

In [ ]:
model = models.build_resnet50_unet(INPUT_SHAPE, NB_CLASSES)

model_name = model.name + '_base'
filepath_check = checkpoint_path + model_name + '/'
model_path = models_path + model_name

mean_iou_metric.reset_states()

# compile
model.compile(optimizer="adam", loss=total_loss, metrics=[mean_iou_metric])

model.summary()

Model: "ResNet50_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                     

In [ ]:
if os.path.exists(model_path):
    model = load_model(model_path)
    training_time = train_times[model_name]

else:
  start_train = time.time()

  mc = ModelCheckpoint(mode='max', filepath=filepath_check, monitor=monitor_val, save_best_only='True', save_weights_only='True', verbose=1)
  es = EarlyStopping(mode='max', monitor=monitor_val, patience=4, verbose=1)

  callbacks = [mc, es]

  # fit
  history = model.fit(data_train, epochs=20, validation_data=data_val, use_multiprocessing=True, workers=2, callbacks=callbacks)

  # Save the model
  model.load_weights(filepath_check)
  model.save(model_path)

  end_train = time.time()
  training_time = end_train - start_train
  train_times[model_name] = training_time

  with open('models/train_times.json', 'w') as f:
    json.dump(train_times, f)

Epoch 1/20
371/371 [==============================] - ETA: 0s - loss: 0.9971 - mean_iou: 0.2892
Epoch 00001: val_mean_iou improved from -inf to 0.21870, saving model to checkpoint/ResNet50_U-Net_base
371/371 [==============================] - 205s 533ms/step - loss: 0.9971 - mean_iou: 0.2892 - val_loss: 1.0954 - val_mean_iou: 0.2187
Epoch 2/20
371/371 [==============================] - ETA: 0s - loss: 1.0636 - mean_iou: 0.2982
Epoch 00002: val_mean_iou did not improve from 0.21870
371/371 [==============================] - 196s 526ms/step - loss: 1.0636 - mean_iou: 0.2982 - val_loss: 1.8558 - val_mean_iou: 0.2039
Epoch 3/20
371/371 [==============================] - ETA: 0s - loss: 1.3190 - mean_iou: 0.3019
Epoch 00003: val_mean_iou improved from 0.21870 to 0.23969, saving model to checkpoint/ResNet50_U-Net_base
371/371 [==============================] - 199s 534ms/step - loss: 1.3190 - mean_iou: 0.3019 - val_loss: 1.8457 - val_mean_iou: 0.2397
Epoch 4/20
371/371 [=====================

In [ ]:
loss_score, mean_iou_score = model.evaluate(data_val)
print("Pour le meilleur modèle on obtient :")
print("mean_iou :", mean_iou_score)
print("loss :", loss_score)

plt.style.use("default")

start_pred = time.time()

y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

end_pred = time.time()
predict_time = end_pred - start_pred

for index in range(10):
    print(index)
    plt.imshow(X_test[index])
    plt.imshow(y_pred_argmax[index],alpha = 0.4)
    plt.show()

synthese[model_name] = helpers.add_model_in_synthese("combine loss", loss_score, mean_iou_score, training_time, predict_time)

Output hidden; open in https://colab.research.google.com to view.

#### modèle DeeplabV3Plus

In [ ]:
mean_iou_metric = dg.Mean_IoU_custom()
mean_iou_metric.reset_states()

model = models.DeeplabV3Plus(256, NB_CLASSES)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[mean_iou_metric])

mc = ModelCheckpoint(mode='max', filepath='/checkpoint/model_deeplab/', monitor='val_mean_iou', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='val_mean_iou', patience=4, verbose=1)

callbacks = [mc, es]

model.fit(data_train, epochs=20, validation_data=data_val, use_multiprocessing=True, workers=2, callbacks=callbacks)

## Synthèse comparative

In [ ]:
pd.DataFrame.from_dict(synthese, orient='index')

,loss type,loss score,mean_iou,Training time,Predict time
U-Net_base,categorical_crossentropy,1.415501,0.221254,2457.389638,2.985835
ResNet50_U-Net_base,categorical_crossentropy,2.100446,0.249706,2091.848141,1.499533
U-Net_dice_loss,dice loss,0.398143,0.194530,1244.418360,0.656268
U-Net_combine_loss,dice loss,3.183513,0.333959,2321.507429,0.733176
U-Net_augmented,dice loss,3.004682,0.264742,2066.903106,0.880152
